In [0]:
# Read the CSV file using Spark
#df_spark = spark.read.csv("/FileStore/tables/uber.csv", header=True, inferSchema=True)

# Convert the Spark DataFrame to a pandas DataFrame
#df_pandas = df_spark.toPandas()

In [0]:
# Display basic information about the DataFrame
#print(df_pandas.info())

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/taxi_trips', recurse=True)

Out[90]: True

# Creating a SQL Table

In [0]:
%sql
DROP TABLE IF EXISTS taxi_trips;
CREATE TABLE taxi_trips (
  VendorID INT,
  tpep_pickup_datetime TIMESTAMP,
  tpep_dropoff_datetime TIMESTAMP,
  passenger_count INT,
  trip_distance FLOAT,
  pickup_longitude FLOAT,
  pickup_latitude FLOAT,
  RatecodeID INT,
  store_and_fwd_flag STRING,
  dropoff_longitude FLOAT,
  dropoff_latitude FLOAT,
  payment_type INT,
  fare_amount FLOAT,
  extra FLOAT,
  mta_tax FLOAT,
  tip_amount FLOAT,
  tolls_amount FLOAT,
  improvement_surcharge FLOAT,
  total_amount FLOAT
);


# Loading Data from CSV file to SQL Table

In [0]:
%sql
-- Load CSV data into a temporary table
CREATE OR REPLACE TEMPORARY VIEW temp_uber_data
USING csv
OPTIONS (
  path 'dbfs:/FileStore/tables/uber.csv',
  header 'true',
  inferSchema 'true'
);

-- Insert data from the temporary table into the taxi_trips table
INSERT INTO taxi_trips
SELECT
  VendorID,
  tpep_pickup_datetime,
  tpep_dropoff_datetime,
  passenger_count,
  trip_distance,
  pickup_longitude,
  pickup_latitude,
  RatecodeID,
  store_and_fwd_flag,
  dropoff_longitude,
  dropoff_latitude,
  payment_type,
  fare_amount,
  extra,
  mta_tax,
  tip_amount,
  tolls_amount,
  improvement_surcharge,
  total_amount
FROM temp_uber_data;


num_affected_rows,num_inserted_rows
100000,100000


# Displaying first 10 Rows

In [0]:
%sql
SELECT * FROM taxi_trips LIMIT 10;

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
1,2016-03-01T00:00:00.000+0000,2016-03-01T00:07:55.000+0000,1,2.5,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.0,0.3,12.35
1,2016-03-01T00:00:00.000+0000,2016-03-01T00:11:06.000+0000,1,2.9,-73.98348,40.767925,1,N,-74.00594,40.733166,1,11.0,0.5,0.5,3.05,0.0,0.3,15.35
2,2016-03-01T00:00:00.000+0000,2016-03-01T00:31:06.000+0000,2,19.98,-73.78202,40.64481,1,N,-73.97454,40.67577,1,54.5,0.5,0.5,8.0,0.0,0.3,63.8
2,2016-03-01T00:00:00.000+0000,2016-03-01T00:00:00.000+0000,3,10.78,-73.86342,40.769814,1,N,-73.96965,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
2,2016-03-01T00:00:00.000+0000,2016-03-01T00:00:00.000+0000,5,30.43,-73.97174,40.792183,3,N,-74.17717,40.695053,1,98.0,0.0,0.0,0.0,15.5,0.3,113.8
2,2016-03-01T00:00:00.000+0000,2016-03-01T00:00:00.000+0000,5,5.92,-74.0172,40.705383,1,N,-73.97807,40.755787,1,23.5,1.0,0.5,5.06,0.0,0.3,30.36
2,2016-03-01T00:00:00.000+0000,2016-03-01T00:00:00.000+0000,6,5.72,-73.99458,40.727848,1,N,0.0,0.0,2,23.0,0.5,0.5,0.0,0.0,0.3,24.3
1,2016-03-01T00:00:01.000+0000,2016-03-01T00:16:04.000+0000,1,6.2,-73.78877,40.64776,1,N,-73.82921,40.712345,3,20.5,0.5,0.5,0.0,0.0,0.3,21.8
1,2016-03-01T00:00:01.000+0000,2016-03-01T00:05:00.000+0000,1,0.7,-73.95822,40.76464,1,N,-73.967896,40.7629,1,5.5,0.5,0.5,2.0,0.0,0.3,8.8
2,2016-03-01T00:00:01.000+0000,2016-03-01T00:24:06.000+0000,3,7.18,-73.98578,40.74119,1,N,-73.94635,40.79788,1,23.5,0.5,0.5,3.2,0.0,0.3,28.0


# Normal Queries

###Q1. Find the total number of trips for each VendorID and the average trip distance for each vendor.

In [0]:
%sql
SELECT VendorID, COUNT(*) AS total_trips, AVG(trip_distance) AS avg_trip_distance
FROM taxi_trips
GROUP BY VendorID;


VendorID,total_trips,avg_trip_distance
1,11673,3.8564550658838104
2,88327,2.9256132325404023


###Q2. Retrieve the trips where the pickup_longitude and pickup_latitude fall within a specific geographical bounding box (e.g., Manhattan).

In [0]:
%sql
SELECT * 
FROM taxi_trips
WHERE pickup_longitude BETWEEN -74.00 AND -73.95
  AND pickup_latitude BETWEEN 40.70 AND 40.85
  LIMIT 10;


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,2016-03-10T12:09:34.000+0000,2016-03-10T12:25:32.000+0000,1,0.79,-73.97768,40.75142,1,N,-73.96891,40.75761,1,10.5,0.0,0.5,2.26,0.0,0.3,13.56
2,2016-03-10T12:09:35.000+0000,2016-03-10T12:33:06.000+0000,2,2.4,-73.99571,40.73891,1,N,-73.98696,40.76433,1,15.5,0.0,0.5,0.0,0.0,0.3,16.3
2,2016-03-10T12:09:35.000+0000,2016-03-10T12:16:00.000+0000,5,0.96,-73.97769,40.74956,1,N,-73.98399,40.737904,1,6.0,0.0,0.5,1.7,0.0,0.3,8.5
2,2016-03-10T12:09:36.000+0000,2016-03-10T12:32:29.000+0000,6,2.69,-73.98179,40.76834,1,N,-73.99035,40.73728,1,15.5,0.0,0.5,3.26,0.0,0.3,19.56
2,2016-03-10T12:09:37.000+0000,2016-03-10T12:18:52.000+0000,2,1.33,-73.96703,40.76683,1,N,-73.95391,40.78219,1,8.0,0.0,0.5,1.0,0.0,0.3,9.8
2,2016-03-10T12:09:37.000+0000,2016-03-10T12:47:02.000+0000,1,4.03,-73.97481,40.741806,1,N,-73.97063,40.785896,1,24.0,0.0,0.5,4.96,0.0,0.3,29.76
2,2016-03-10T12:09:37.000+0000,2016-03-10T12:13:21.000+0000,1,0.38,-73.998535,40.7136,1,N,-74.004234,40.716015,2,4.5,0.0,0.5,0.0,0.0,0.3,5.3
2,2016-03-10T12:09:38.000+0000,2016-03-10T12:12:06.000+0000,1,0.63,-73.95909,40.815033,1,N,-73.959656,40.808716,2,4.0,0.0,0.5,0.0,0.0,0.3,4.8
2,2016-03-10T12:09:38.000+0000,2016-03-10T12:13:43.000+0000,3,0.64,-73.99911,40.744713,1,N,-74.002304,40.7503,1,5.0,0.0,0.5,1.16,0.0,0.3,6.96
2,2016-03-10T12:09:38.000+0000,2016-03-10T12:17:44.000+0000,4,1.51,-73.962524,40.77582,1,N,-73.98126,40.778423,2,7.5,0.0,0.5,0.0,0.0,0.3,8.3


###Q3. Find the top 5 longest trips (in terms of trip_distance) that were paid in cash (payment_type = 2).

In [0]:
%sql
SELECT * 
FROM taxi_trips
WHERE payment_type = 2
ORDER BY trip_distance DESC
LIMIT 5;


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
1,2016-03-01T01:02:58.000+0000,2016-03-01T03:54:52.000+0000,1,160.8,-73.90559,40.874123,4,N,-73.71071,42.666893,2,819.5,0.5,0.5,0.0,12.0,0.3,832.8
2,2016-03-10T13:59:29.000+0000,2016-03-10T16:25:37.000+0000,1,55.01,-73.98124,40.75799,5,N,-73.978355,40.7568,2,200.0,0.0,0.0,0.0,5.54,0.3,205.84
2,2016-03-10T10:18:12.000+0000,2016-03-10T18:32:00.000+0000,2,51.27,-73.94728,40.78394,1,N,-73.95114,40.78518,2,314.5,0.0,0.5,0.0,0.0,0.3,315.3
2,2016-03-10T11:37:01.000+0000,2016-03-10T13:54:55.000+0000,1,49.56,-73.995636,40.725056,1,N,-73.99563,40.724884,2,155.0,0.0,0.5,0.0,11.08,0.3,166.88
2,2016-03-10T08:59:54.000+0000,2016-03-10T12:11:54.000+0000,1,48.18,-73.776726,40.64535,2,N,-73.806946,40.68551,2,52.0,0.0,0.5,0.0,0.0,0.3,52.8


###Q4. Calculate the total revenue (total_amount) generated per day.

In [0]:
%sql
SELECT DATE(tpep_pickup_datetime) AS pickup_date, SUM(total_amount) AS total_revenue
FROM taxi_trips
GROUP BY DATE(tpep_pickup_datetime)
ORDER BY pickup_date;


pickup_date,total_revenue
2016-03-01,405907.2987920642
2016-03-10,1233164.7891432047


###Q5. Identify the trips where the tip_amount is greater than 20% of the fare_amount.

In [0]:
%sql
SELECT * 
FROM taxi_trips
WHERE tip_amount > 0.2 * fare_amount
LIMIT 10;


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,2016-03-10T12:09:34.000+0000,2016-03-10T12:25:32.000+0000,1,0.79,-73.97768,40.75142,1,N,-73.96891,40.75761,1,10.5,0.0,0.5,2.26,0.0,0.3,13.56
2,2016-03-10T12:09:35.000+0000,2016-03-10T12:16:00.000+0000,5,0.96,-73.97769,40.74956,1,N,-73.98399,40.737904,1,6.0,0.0,0.5,1.7,0.0,0.3,8.5
2,2016-03-10T12:09:36.000+0000,2016-03-10T12:32:29.000+0000,6,2.69,-73.98179,40.76834,1,N,-73.99035,40.73728,1,15.5,0.0,0.5,3.26,0.0,0.3,19.56
2,2016-03-10T12:09:36.000+0000,2016-03-10T12:25:57.000+0000,1,1.91,-74.00588,40.72489,1,N,-73.98872,40.73673,1,12.0,0.0,0.5,2.56,0.0,0.3,15.36
2,2016-03-10T12:09:37.000+0000,2016-03-10T12:22:44.000+0000,5,1.64,-74.00679,40.74871,1,N,-73.99342,40.73459,1,10.0,0.0,0.5,2.16,0.0,0.3,12.96
2,2016-03-10T12:09:37.000+0000,2016-03-10T12:47:02.000+0000,1,4.03,-73.97481,40.741806,1,N,-73.97063,40.785896,1,24.0,0.0,0.5,4.96,0.0,0.3,29.76
2,2016-03-10T12:09:38.000+0000,2016-03-10T12:13:43.000+0000,3,0.64,-73.99911,40.744713,1,N,-74.002304,40.7503,1,5.0,0.0,0.5,1.16,0.0,0.3,6.96
2,2016-03-10T12:09:39.000+0000,2016-03-10T12:14:27.000+0000,2,0.9,-73.99053,40.7187,1,N,-73.98977,40.728622,1,5.5,0.0,0.5,1.58,0.0,0.3,7.88
2,2016-03-10T12:09:39.000+0000,2016-03-10T12:32:20.000+0000,1,2.51,-73.980255,40.742622,1,N,-73.95927,40.742336,1,16.0,0.0,0.5,4.47,5.54,0.3,26.81
2,2016-03-10T12:09:39.000+0000,2016-03-10T12:25:51.000+0000,1,3.02,-73.962204,40.7792,1,N,-73.964165,40.80747,1,13.5,0.0,0.5,2.86,0.0,0.3,17.16


###Q6. Find the most common RatecodeID for trips with more than 4 passengers.

In [0]:
%sql
SELECT RatecodeID, COUNT(*) AS count
FROM taxi_trips
WHERE passenger_count > 4
GROUP BY RatecodeID
ORDER BY count DESC
LIMIT 1;


RatecodeID,count
1,14453


###Q7. Determine the average trip distance and total fare amount for each hour of the day.

In [0]:
%sql
SELECT HOUR(tpep_pickup_datetime) AS pickup_hour, AVG(trip_distance) AS avg_trip_distance, SUM(total_amount) AS total_fare
FROM taxi_trips
GROUP BY HOUR(tpep_pickup_datetime)
ORDER BY pickup_hour;


pickup_hour,avg_trip_distance,total_fare
0,3.8484178543299215,123377.42957812548
1,3.757442145054691,70962.12978708744
2,3.4637701740276516,41108.14986854792
3,3.892290328560217,31724.87986242771
4,4.754800415107813,38209.58984684944
5,4.165872330395699,68055.14984869957
6,3.91125984055985,32469.970000326633
7,2.773499476292917,142396.04996085167
8,2.5475136651051384,177146.46985912323
9,2.6049337067023046,167358.04982948303


###Q8. Find the maximum tolls_amount paid for each VendorID and the corresponding trip details.

In [0]:
%sql
SELECT a.*
FROM taxi_trips a
JOIN (
  SELECT VendorID, MAX(tolls_amount) AS max_tolls
  FROM taxi_trips
  GROUP BY VendorID
) b
ON a.VendorID = b.VendorID AND a.tolls_amount = b.max_tolls;


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,2016-03-10T11:35:11.000+0000,2016-03-10T12:25:23.000+0000,1,32.39,-73.88532,40.77311,3,N,-74.183,40.68792,1,103.0,0.0,0.0,10.0,25.54,0.3,138.84
1,2016-03-01T02:20:38.000+0000,2016-03-01T02:36:09.000+0000,1,5.9,-73.97132,40.757107,5,N,-74.034035,40.73008,1,55.0,0.0,0.0,0.0,21.0,0.3,76.3
1,2016-03-01T03:43:09.000+0000,2016-03-01T04:21:29.000+0000,1,20.4,-73.973114,40.756805,3,N,-74.17724,40.695156,2,72.5,0.5,0.0,0.0,21.0,0.3,94.3
1,2016-03-01T04:52:53.000+0000,2016-03-01T05:16:20.000+0000,1,18.6,-73.98659,40.750618,3,N,-74.18183,40.687687,1,67.5,0.5,0.0,10.0,21.0,0.3,99.3


###Q9. Identify the days of the week (e.g., Monday, Tuesday) with the highest average total_amount.

In [0]:
%sql
SELECT DAYOFWEEK(tpep_pickup_datetime) AS day_of_week, AVG(total_amount) AS avg_total_amount
FROM taxi_trips
GROUP BY DAYOFWEEK(tpep_pickup_datetime)
ORDER BY avg_total_amount DESC;


day_of_week,avg_total_amount
3,17.480934487169
5,16.061015748153228


###Q10. Calculate the average trip_distance and total_amount for each combination of VendorID and RatecodeID.

In [0]:
%sql
SELECT VendorID, RatecodeID, AVG(trip_distance) AS avg_trip_distance, AVG(total_amount) AS avg_total_amount
FROM taxi_trips
GROUP BY VendorID, RatecodeID
ORDER BY VendorID, RatecodeID;



VendorID,RatecodeID,avg_trip_distance,avg_total_amount
1,1,3.376783165818986,15.239184170259898
1,2,17.665765760114063,64.86062943971193
1,3,14.653658494716737,79.40731792915159
1,4,28.609090978449043,133.39090806787664
1,5,4.846478860143205,65.15408511816615
1,6,0.10000000149011612,3.299999952316284
2,1,2.5592361277935765,14.837183957659589
2,2,17.407353267350942,64.9673799557518
2,3,16.762262409047235,90.23520467076366
2,4,17.699729597246325,76.52513534958298


###Q11. Find the trip with the longest duration (tpep_dropoff_datetime - tpep_pickup_datetime) and display the trip details.

In [0]:
%sql
SELECT *, 
       (tpep_dropoff_datetime - tpep_pickup_datetime) AS trip_duration
FROM taxi_trips
ORDER BY trip_duration DESC
LIMIT 1;


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_duration
2,2016-03-10T12:20:08.000+0000,2016-03-11T12:19:31.000+0000,1,3.62,-73.984146,40.755177,1,N,-74.00814,40.72996,1,21.5,0.0,0.5,0.0,0.0,0.3,22.3,"Map(zero -> false, seconds -> 86363, negative -> false, nano -> 0, units -> List(SECONDS, NANOS))"


###Q12. Find the average trip distance and total amount for trips that had more than 2 passengers, grouped by payment_type and ordered by average trip distance in descending order.

In [0]:
%sql
SELECT 
    payment_type,
    AVG(trip_distance) AS average_trip_distance,
    SUM(total_amount) AS total_amount
FROM 
    taxi_trips
WHERE 
    passenger_count > 2
GROUP BY 
    payment_type
ORDER BY 
    average_trip_distance DESC;


payment_type,average_trip_distance,total_amount
1,3.0786502921782284,246347.59947824478
2,2.5354649620282528,88042.1201953888
4,0.9341666760543982,-40.21999955177307
3,0.8070000041276216,-12.299999952316284


###Q13. Identify the top 5 longest trips (by trip distance) that used a RatecodeID of 1, and display their pickup and dropoff locations, along with the trip distance.

In [0]:
%sql
SELECT 
    pickup_longitude,
    pickup_latitude,
    dropoff_longitude,
    dropoff_latitude,
    trip_distance
FROM 
    taxi_trips
WHERE 
    RatecodeID = 1
ORDER BY 
    trip_distance DESC
LIMIT 5;


pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance
-73.94728,40.78394,-73.95114,40.78518,51.27
-73.995636,40.725056,-73.99563,40.724884,49.56
-73.871254,40.773945,-74.33979,40.519493,41.55
-74.009636,40.72606,-74.351456,40.50695,34.95
-73.96937,40.7624,-73.52637,40.76681,33.76


###Q14. Find the average fare amount, total improvement surcharge, and number of trips for each combination of payment_type and RatecodeID, where the trips had a tip amount greater than $5.

In [0]:
%sql
SELECT 
    payment_type,
    RatecodeID,
    AVG(fare_amount) AS average_fare_amount,
    SUM(improvement_surcharge) AS total_improvement_surcharge,
    COUNT(*) AS number_of_trips
FROM 
    taxi_trips
WHERE 
    tip_amount > 5
GROUP BY 
    payment_type, 
    RatecodeID;


payment_type,RatecodeID,average_fare_amount,total_improvement_surcharge,number_of_trips
1,2,52.0,434.1000172495842,1447
1,1,31.149317406143346,1758.0000698566437,5860
1,3,68.49714285714286,52.50000208616257,175
1,4,72.22727272727273,6.600000262260437,22
3,2,52.0,0.30000001192092896,1
1,5,82.58308321634928,35.40000140666962,120


###Q15. Determine the average trip duration (in minutes) and average total amount for trips that started and ended in the same borough (considering approximate longitudes and latitudes), grouped by passenger_count and ordered by average trip duration in ascending order. Assume pickup and dropoff coordinates are within 0.01 degrees of each other to be considered in the same borough.

In [0]:
%sql
SELECT 
    passenger_count,
    AVG(TIMESTAMPDIFF(MINUTE, tpep_pickup_datetime, tpep_dropoff_datetime)) AS average_trip_duration,
    AVG(total_amount) AS average_total_amount
FROM 
    taxi_trips
WHERE 
    ABS(pickup_longitude - dropoff_longitude) <= 0.01 AND
    ABS(pickup_latitude - dropoff_latitude) <= 0.01
GROUP BY 
    passenger_count
ORDER BY 
    average_trip_duration ASC;



passenger_count,average_trip_duration,average_total_amount
6,7.9094922737306845,7.846306201023947
2,7.939454424484365,8.364454520914608
5,8.309319899244333,8.028075656722715
1,8.455956025605344,8.471278966970342
4,9.004555808656036,7.694965921936383
3,11.912017167381974,8.468658881126043
0,140.66666666666666,37.96666590372721


# Nested Queries

###Q16. List the top 5 days with the highest total tip amount, along with the total tip amount for those days.

In [0]:
%sql
SELECT pickup_date, total_tip
FROM (
    SELECT DATE(tpep_pickup_datetime) AS pickup_date, SUM(tip_amount) AS total_tip
    FROM taxi_trips
    GROUP BY pickup_date
    ORDER BY total_tip DESC
    LIMIT 5
) AS top_tip_days;


pickup_date,total_tip
2016-03-10,144818.639970541
2016-03-01,42434.45999929495


###Q17. Find the maximum trip distance for trips where the fare amount is greater than the average fare amount of trips that have a passenger count of 2.

In [0]:
%sql
SELECT MAX(trip_distance) AS max_distance
FROM taxi_trips
WHERE fare_amount > (
    SELECT AVG(fare_amount)
    FROM taxi_trips
    WHERE passenger_count = 2
);

max_distance
184.4


###Q18. Calculate the total amount collected on days when the total number of trips was greater than the average number of trips per day.

In [0]:
%sql
SELECT SUM(total_amount) AS total_collected
FROM taxi_trips
WHERE DATE(tpep_pickup_datetime) IN (
    SELECT trip_date
    FROM (
        SELECT DATE(tpep_pickup_datetime) AS trip_date, COUNT(*) AS trip_count
        FROM taxi_trips
        GROUP BY trip_date
    ) AS daily_trips
    WHERE trip_count > (
        SELECT AVG(trip_count)
        FROM (
            SELECT DATE(tpep_pickup_datetime) AS trip_date, COUNT(*) AS trip_count
            FROM taxi_trips
            GROUP BY trip_date
        ) AS avg_daily_trips
    )
);


total_collected
1233164.7891432047


###Q19. Determine the average trip distance for trips that started and ended in different rate zones.

In [0]:
%sql
WITH pickup_ratecodes AS (
    SELECT tpep_pickup_datetime, RatecodeID AS pickup_ratecode
    FROM taxi_trips
),
dropoff_ratecodes AS (
    SELECT tpep_dropoff_datetime, RatecodeID AS dropoff_ratecode
    FROM taxi_trips
)
SELECT AVG(trip_distance) AS average_distance
FROM taxi_trips tt
JOIN pickup_ratecodes pr ON tt.tpep_pickup_datetime = pr.tpep_pickup_datetime
JOIN dropoff_ratecodes dr ON tt.tpep_dropoff_datetime = dr.tpep_dropoff_datetime
WHERE pr.pickup_ratecode != dr.dropoff_ratecode;



average_distance
5.7421390817644955


###Q20. Find the top 3 longest trips in terms of distance where the total amount paid was more than twice the average total amount for all trips. List the trip distance, total amount, and the date of the trip.

In [0]:
%sql
WITH avg_total_amount AS (
    SELECT AVG(total_amount) AS avg_amount
    FROM taxi_trips
)
SELECT trip_distance, total_amount, tpep_pickup_datetime
FROM taxi_trips
WHERE total_amount > 2 * (SELECT avg_amount FROM avg_total_amount)
ORDER BY trip_distance DESC
LIMIT 3;


trip_distance,total_amount,tpep_pickup_datetime
184.4,58.34,2016-03-01T06:14:14.000+0000
160.8,832.8,2016-03-01T01:02:58.000+0000
55.01,205.84,2016-03-10T13:59:29.000+0000


# Window Functions

###Q21. Find the average trip distance for each vendor on a daily basis. Include only those days where the vendor has at least 5 trips.



In [0]:
%sql
WITH daily_trip_counts AS (
  SELECT 
    VendorID, 
    DATE(tpep_pickup_datetime) AS trip_date, 
    COUNT(*) AS trip_count
  FROM taxi_trips
  GROUP BY VendorID, DATE(tpep_pickup_datetime)
),
daily_avg_distance AS (
  SELECT 
    VendorID, 
    DATE(tpep_pickup_datetime) AS trip_date, 
    AVG(trip_distance) AS avg_distance
  FROM taxi_trips
  GROUP BY VendorID, DATE(tpep_pickup_datetime)
)
SELECT 
  d.VendorID, 
  d.trip_date, 
  d.avg_distance
FROM daily_avg_distance d
JOIN daily_trip_counts c
  ON d.VendorID = c.VendorID AND d.trip_date = c.trip_date
WHERE c.trip_count >= 5;


VendorID,trip_date,avg_distance
2,2016-03-10,2.765324823648673
1,2016-03-01,3.8564550658838104
2,2016-03-01,3.9914263471768416


###Q22. Calculate the cumulative total fare amount for each vendor, ordered by the pickup datetime.

In [0]:
%sql
SELECT 
  VendorID, 
  tpep_pickup_datetime, 
  fare_amount, 
  SUM(fare_amount) OVER (PARTITION BY VendorID ORDER BY tpep_pickup_datetime) AS cumulative_fare
FROM taxi_trips
LIMIT 10;


VendorID,tpep_pickup_datetime,fare_amount,cumulative_fare
1,2016-03-01T00:00:00.000+0000,9.0,20.0
1,2016-03-01T00:00:00.000+0000,11.0,20.0
1,2016-03-01T00:00:01.000+0000,20.5,46.0
1,2016-03-01T00:00:01.000+0000,5.5,46.0
1,2016-03-01T00:00:02.000+0000,8.0,59.5
1,2016-03-01T00:00:02.000+0000,5.5,59.5
1,2016-03-01T00:00:03.000+0000,11.0,70.5
1,2016-03-01T00:00:04.000+0000,4.5,109.0
1,2016-03-01T00:00:04.000+0000,5.5,109.0
1,2016-03-01T00:00:04.000+0000,28.5,109.0


###Q23. Find the rank of each trip based on the total amount spent within each vendor, ordered by pickup datetime. The most expensive trip should have the rank 1.

In [0]:
%sql
SELECT 
  VendorID, 
  tpep_pickup_datetime, 
  total_amount,
  RANK() OVER (PARTITION BY VendorID ORDER BY total_amount DESC, tpep_pickup_datetime) AS rank
FROM taxi_trips
LIMIT 10;


VendorID,tpep_pickup_datetime,total_amount,rank
1,2016-03-01T01:02:58.000+0000,832.8,1
1,2016-03-01T02:10:47.000+0000,312.85,2
1,2016-03-01T01:45:24.000+0000,257.79,3
1,2016-03-01T00:47:51.000+0000,216.8,4
1,2016-03-01T00:10:03.000+0000,170.3,5
1,2016-03-01T00:38:10.000+0000,163.3,6
1,2016-03-01T01:35:25.000+0000,150.3,7
1,2016-03-01T03:58:35.000+0000,150.3,8
1,2016-03-01T01:09:26.000+0000,145.3,9
1,2016-03-01T00:18:00.000+0000,118.34,10


###Q24. Calculate the running average of tip amounts for each vendor over the course of the trips, ordered by pickup datetime.

In [0]:
%sql
SELECT 
  VendorID, 
  tpep_pickup_datetime, 
  tip_amount,
  AVG(tip_amount) OVER (PARTITION BY VendorID ORDER BY tpep_pickup_datetime ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_avg_tip
FROM taxi_trips
LIMIT 10;


VendorID,tpep_pickup_datetime,tip_amount,running_avg_tip
1,2016-03-01T00:00:00.000+0000,2.05,2.049999952316284
1,2016-03-01T00:00:00.000+0000,3.05,2.549999952316284
1,2016-03-01T00:00:01.000+0000,0.0,1.6999999682108562
1,2016-03-01T00:00:01.000+0000,2.0,1.774999976158142
1,2016-03-01T00:00:02.000+0000,0.0,1.4199999809265136
1,2016-03-01T00:00:02.000+0000,2.2,1.549999992052714
1,2016-03-01T00:00:03.000+0000,0.0,1.328571421759469
1,2016-03-01T00:00:04.000+0000,1.15,1.3062499910593033
1,2016-03-01T00:00:04.000+0000,0.0,1.161111103163825
1,2016-03-01T00:00:04.000+0000,0.0,1.0449999928474427


###Q25. For each trip, calculate the percentage of the total fare that is made up by the tip amount, partitioned by the day of the trip.

In [0]:
%sql
SELECT 
  VendorID, 
  tpep_pickup_datetime, 
  total_amount, 
  tip_amount, 
  (tip_amount / total_amount) * 100 AS tip_percentage,
  AVG((tip_amount / total_amount) * 100) OVER (PARTITION BY DATE(tpep_pickup_datetime)) AS daily_avg_tip_percentage
FROM taxi_trips
LIMIT 10;


VendorID,tpep_pickup_datetime,total_amount,tip_amount,tip_percentage,daily_avg_tip_percentage
1,2016-03-01T00:01:06.000+0000,8.8,0.0,0.0,9.281809092060248
2,2016-03-01T00:01:06.000+0000,7.3,0.0,0.0,9.281809092060248
2,2016-03-01T00:01:06.000+0000,70.01,11.67,16.66904687930965,9.281809092060248
1,2016-03-01T00:01:07.000+0000,10.56,1.76,16.6666659140828,9.281809092060248
1,2016-03-01T00:01:07.000+0000,6.95,1.15,16.546762700986868,9.281809092060248
2,2016-03-01T00:01:07.000+0000,8.76,1.46,16.666666666666664,9.281809092060248
2,2016-03-01T00:01:07.000+0000,7.88,1.58,20.05076167473437,9.281809092060248
2,2016-03-01T00:01:07.000+0000,11.8,1.0,8.474576134203634,9.281809092060248
1,2016-03-01T00:01:08.000+0000,59.34,0.0,0.0,9.281809092060248
1,2016-03-01T00:01:08.000+0000,66.33,7.99,12.0458307712233,9.281809092060248


# Common Table Expressions

###Q26. Calculate the Total Revenue per VendorID for Vendors with More than 100 Trips

In [0]:
%sql
WITH VendorTripCounts AS (
    SELECT VendorID, COUNT(*) AS trip_count
    FROM taxi_trips
    GROUP BY VendorID
)
SELECT t.VendorID, SUM(t.total_amount) AS total_revenue
FROM taxi_trips t
JOIN VendorTripCounts v ON t.VendorID = v.VendorID
WHERE v.trip_count > 100
GROUP BY t.VendorID;


VendorID,total_revenue
1,201873.51932424307
2,1437198.5686110258


###Q27. Find the VendorID with the Highest Average Tip Percentage

In [0]:
%sql
WITH TipPercentages AS (
    SELECT VendorID, 
           CASE WHEN total_amount > 0 THEN (tip_amount / total_amount) * 100 ELSE 0 END AS tip_percentage
    FROM taxi_trips
)
SELECT VendorID, AVG(tip_percentage) AS avg_tip_percentage
FROM TipPercentages
GROUP BY VendorID
ORDER BY avg_tip_percentage DESC
LIMIT 1;


VendorID,avg_tip_percentage
2,10.48184183880434


###Q28. dentify the Top 10 Longest Trips (in miles) for Each RatecodeID

In [0]:
%sql
WITH RankedTrips AS (
    SELECT *,
           ROW_NUMBER() OVER(PARTITION BY RatecodeID ORDER BY trip_distance DESC) AS trip_rank
    FROM taxi_trips
)
SELECT *
FROM RankedTrips
WHERE trip_rank <= 10
LIMIT 10;


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_rank
2,2016-03-10T10:18:12.000+0000,2016-03-10T18:32:00.000+0000,2,51.27,-73.94728,40.78394,1,N,-73.95114,40.78518,2,314.5,0.0,0.5,0.0,0.0,0.3,315.3,1
2,2016-03-10T11:37:01.000+0000,2016-03-10T13:54:55.000+0000,1,49.56,-73.995636,40.725056,1,N,-73.99563,40.724884,2,155.0,0.0,0.5,0.0,11.08,0.3,166.88,2
2,2016-03-01T00:13:09.000+0000,2016-03-01T01:18:39.000+0000,1,41.55,-73.871254,40.773945,1,N,-74.33979,40.519493,1,110.5,0.5,0.5,38.35,16.04,0.3,166.19,3
2,2016-03-10T09:12:57.000+0000,2016-03-10T10:10:59.000+0000,2,34.95,-74.009636,40.72606,1,N,-74.351456,40.50695,1,95.0,0.0,0.5,8.0,12.5,0.3,116.3,4
2,2016-03-01T02:21:50.000+0000,2016-03-01T03:15:02.000+0000,2,33.76,-73.96937,40.7624,1,N,-73.52637,40.76681,2,89.5,0.5,0.5,0.0,0.0,0.3,90.8,5
2,2016-03-01T04:48:29.000+0000,2016-03-01T05:49:40.000+0000,1,33.18,-73.97517,40.760204,1,N,-73.861694,40.76835,1,92.5,0.5,0.5,0.0,11.08,0.3,104.88,6
2,2016-03-10T13:33:32.000+0000,2016-03-10T14:44:57.000+0000,2,33.11,-73.86281,40.76874,1,N,-73.8617,40.76834,1,92.0,0.0,0.5,10.0,11.08,0.3,113.88,7
2,2016-03-10T14:19:47.000+0000,2016-03-10T15:52:35.000+0000,1,32.8,-73.9877,40.770226,1,N,-73.73474,40.62336,1,98.0,0.0,0.5,20.87,5.54,0.3,125.21,8
1,2016-03-01T00:10:06.000+0000,2016-03-01T00:57:57.000+0000,1,32.6,-73.9898,40.75649,1,N,-74.06666,41.116085,2,86.5,0.5,0.5,0.0,10.5,0.3,98.3,9
1,2016-03-01T00:18:00.000+0000,2016-03-01T01:01:49.000+0000,1,31.6,-73.79008,40.643265,1,N,-73.81456,41.00407,1,82.5,0.5,0.5,29.0,5.54,0.3,118.34,10


###Q29. Calculate the Hourly Average Fare Amount and Tip Amount Over the Day

In [0]:
%sql
WITH HourlyAverages AS (
    SELECT DATE_TRUNC('hour', tpep_pickup_datetime) AS pickup_hour,
           AVG(fare_amount) AS avg_fare_amount,
           AVG(tip_amount) AS avg_tip_amount
    FROM taxi_trips
    GROUP BY pickup_hour
    ORDER BY pickup_hour
)
SELECT pickup_hour,
       avg_fare_amount,
       avg_tip_amount
FROM HourlyAverages;


pickup_hour,avg_fare_amount,avg_tip_amount
2016-03-01T00:00:00.000+0000,14.008186184430027,1.906209916669177
2016-03-01T01:00:00.000+0000,13.907299903590545,1.7467695291049907
2016-03-01T02:00:00.000+0000,12.901617986164403,1.4934281322276142
2016-03-01T03:00:00.000+0000,14.090516130113473,1.4285322557774283
2016-03-01T04:00:00.000+0000,16.071228616654164,1.8768636600494368
2016-03-01T05:00:00.000+0000,14.440221801460643,2.0858506889480264
2016-03-01T06:00:00.000+0000,13.584519685039252,2.0052335954791918
2016-03-10T07:00:00.000+0000,12.006950785152576,1.8155089002800862
2016-03-10T08:00:00.000+0000,12.213717116485247,1.8552271950071049
2016-03-10T09:00:00.000+0000,12.594820727643068,1.889086834007368


###Q30. Find the Most Common Pickup and Dropoff Locations

In [0]:
%sql
WITH PickupLocations AS (
    SELECT pickup_longitude, pickup_latitude, COUNT(*) AS pickup_count
    FROM taxi_trips
    GROUP BY pickup_longitude, pickup_latitude
    ORDER BY pickup_count DESC
    LIMIT 1
),
DropoffLocations AS (
    SELECT dropoff_longitude, dropoff_latitude, COUNT(*) AS dropoff_count
    FROM taxi_trips
    GROUP BY dropoff_longitude, dropoff_latitude
    ORDER BY dropoff_count DESC
    LIMIT 1
)
SELECT 'Most Common Pickup Location:', pickup_longitude, pickup_latitude, pickup_count FROM PickupLocations
UNION ALL
SELECT 'Most Common Dropoff Location:', dropoff_longitude, dropoff_latitude, dropoff_count FROM DropoffLocations;


Most Common Pickup Location:,pickup_longitude,pickup_latitude,pickup_count
Most Common Pickup Location:,0.0,0.0,925
Most Common Dropoff Location:,0.0,0.0,893


# Creating STAR Schema Dimensional Model

In [0]:
# Define the path to your image
image_path = "/dbfs/FileStore/tables/Star_Schema_Dimensional_Model.png"

# Display the image using displayHTML
displayHTML(f'<img src="files/tables/Star_Schema_Dimensional_Model.png" alt="Star Schema Dimensional Model" style="width: 900px;"/>')




## Creating and Inserting Data into Fact Table

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/fact_taxi_trips', recurse=True)

Out[125]: True

In [0]:
%sql

DROP TABLE IF EXISTS fact_taxi_trips;
CREATE TABLE fact_taxi_trips (
  trip_id INT,
  VendorID INT,
  pickup_datetime_id INT,
  dropoff_datetime_id INT,
  passenger_count INT,
  trip_distance FLOAT,
  pickup_location_id INT,
  dropoff_location_id INT,
  RatecodeID INT,
  payment_type_id INT,
  fare_amount FLOAT,
  extra FLOAT,
  mta_tax FLOAT,
  tip_amount FLOAT,
  tolls_amount FLOAT,
  improvement_surcharge FLOAT,
  total_amount FLOAT
);

In [0]:
%sql
INSERT INTO fact_taxi_trips (
  trip_id,
  VendorID,
  pickup_datetime_id,
  dropoff_datetime_id,
  passenger_count,
  trip_distance,
  pickup_location_id,
  dropoff_location_id,
  RatecodeID,
  payment_type_id,
  fare_amount,
  extra,
  mta_tax,
  tip_amount,
  tolls_amount,
  improvement_surcharge,
  total_amount
)
SELECT 
  ROW_NUMBER() OVER (ORDER BY tpep_pickup_datetime) AS trip_id,
  VendorID,
  CAST(UNIX_TIMESTAMP(tpep_pickup_datetime) AS INT) AS pickup_datetime_id,
  CAST(UNIX_TIMESTAMP(tpep_dropoff_datetime) AS INT) AS dropoff_datetime_id,
  passenger_count,
  trip_distance,
  CAST(pickup_longitude * 1000000 + pickup_latitude * 1000000 AS INT) AS pickup_location_id,
  CAST(dropoff_longitude * 1000000 + dropoff_latitude * 1000000 AS INT) AS dropoff_location_id,
  RatecodeID,
  payment_type AS payment_type_id,
  fare_amount,
  extra,
  mta_tax,
  tip_amount,
  tolls_amount,
  improvement_surcharge,
  total_amount
FROM taxi_trips;


num_affected_rows,num_inserted_rows
100000,100000


## Creating Dimensions Tables

In [0]:
%sql
DROP TABLE IF EXISTS dim_time;
CREATE TABLE dim_time (
  time_id INT,
  datetime TIMESTAMP,
  year INT,
  month INT,
  day INT,
  hour INT,
  minute INT,
  second INT
);

-- Extract unique pickup and dropoff datetime values
INSERT INTO dim_time (time_id, datetime, year, month, day, hour, minute, second)
SELECT DISTINCT
  CAST(UNIX_TIMESTAMP(datetime) AS INT) AS time_id,
  datetime,
  YEAR(datetime) AS year,
  MONTH(datetime) AS month,
  DAY(datetime) AS day,
  HOUR(datetime) AS hour,
  MINUTE(datetime) AS minute,
  SECOND(datetime) AS second
FROM (
  SELECT tpep_pickup_datetime AS datetime FROM taxi_trips
  UNION
  SELECT tpep_dropoff_datetime AS datetime FROM taxi_trips
) AS unique_datetimes;


num_affected_rows,num_inserted_rows
47007,47007


In [0]:
%sql
DROP TABLE IF EXISTS dim_location;
CREATE TABLE dim_location (
  location_id INT,
  longitude FLOAT,
  latitude FLOAT
);

-- Extract unique pickup and dropoff locations
INSERT INTO dim_location (location_id, longitude, latitude)
SELECT DISTINCT 
  CAST(pickup_longitude * 1000000 + pickup_latitude * 1000000 AS INT) AS location_id,
  pickup_longitude AS longitude,
  pickup_latitude AS latitude
FROM taxi_trips
UNION
SELECT DISTINCT 
  CAST(dropoff_longitude * 1000000 + dropoff_latitude * 1000000 AS INT) AS location_id,
  dropoff_longitude AS longitude,
  dropoff_latitude AS latitude
FROM taxi_trips;


num_affected_rows,num_inserted_rows
194957,194957


In [0]:
# Remove the directories if they exist
dbutils.fs.rm("dbfs:/user/hive/warehouse/rate_code_dim", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/trip_distance_dim", recurse=True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/payment_type_dim", recurse=True)

Out[138]: False

In [0]:
%sql

DROP TABLE IF EXISTS trip_distance_dim;
DROP TABLE IF EXISTS rate_code_dim;
DROP TABLE IF EXISTS payment_type_dim;

CREATE TABLE trip_distance_dim (
  trip_distance_id INT,
  trip_distance FLOAT
);


CREATE TABLE rate_code_dim (
  rate_code_id INT,
  rate_code_name STRING
);


CREATE TABLE payment_type_dim (
  payment_type_id INT,
  payment_type_name STRING
);



In [0]:
%sql
-- Populate trip_distance_dim
INSERT INTO trip_distance_dim (trip_distance_id, trip_distance)
SELECT DISTINCT NULL, trip_distance
FROM fact_taxi_trips;

-- Populate rate_code_dim (assuming predefined mappings)
INSERT INTO rate_code_dim (rate_code_id, rate_code_name)
VALUES
  (1, 'Standard Rate'),
  (2, 'JFK'),
  (3, 'Newark'),
  (4, 'Nassau or Westchester'),
  (5, 'Negotiated fare'),
  (6, 'Group ride');

-- Populate payment_type_dim (assuming predefined mappings)
INSERT INTO payment_type_dim (payment_type_id, payment_type_name)
VALUES
  (1, 'Credit card'),
  (2, 'Cash'),
  (3, 'No charge'),
  (4, 'Dispute'),
  (5, 'Unknown'),
  (6, 'Voided trip');


num_affected_rows,num_inserted_rows
6,6
